In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
base = '/Users/eddie/Downloads/2024_SMT_Data_Challenge/2024_SMT_Data_Challenge'

In [3]:
team_info = base + '/team_info.csv'
game_info = base + '/game_info'
game_events = base + '/game_events'
player_pos = base + '/player_pos'
ball_pos = base + '/ball_pos'

In [4]:
def combine(directory):
    df_list = []
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            name = directory.split('/')[-1]
            if file == f'{name}.csv':
                path = os.path.join(subdir, file)

                df = pd.read_csv(path)
                df_list.append(df)
    return df_list
                

In [5]:
df_team_info = pd.read_csv(team_info)
df_game_info = pd.concat(combine(game_info), ignore_index = True)
df_game_events = pd.concat(combine(game_events), ignore_index = True)
df_player_pos = combine(player_pos)
df_ball_pos = combine(ball_pos)


In [6]:
df_game_info.replace('\\N', np.nan, inplace=True)  
numeric_columns = df_game_info.columns[df_game_info.columns.get_loc('pitcher'):]  
df_game_info[numeric_columns] = df_game_info[numeric_columns].astype(float)


/var/folders/0m/p03jcv9j4td3thddmd99m0300000gp/T/ipykernel_42647/3863594838.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_game_info.replace('\\N', np.nan, inplace=True)


In [7]:
df_team_info.to_parquet('Combined/team_info.parquet', index = False)
df_game_info.to_parquet('Combined/game_info.parquet', index = False)
df_game_events.to_parquet('Combined/game_events.parquet', index = False)

# Manually organize each separate game into the file
for i, item in enumerate(df_player_pos):
    pd.DataFrame(item).to_parquet(f'Combined/player_pos/{item.game_str[i]}', index = False)

for i, item in enumerate(df_ball_pos):
    pd.DataFrame(item).to_parquet(f'Combined/ball_pos/{item.game_str[i]}', index = False)